## Deployment

Какво е deployment? В изолация, всяко парче код е безполезно, включително и ML/AI. Deployment е процесът по "отваряне" на кода, така че други системи или потребители да могат да достъпват нашия софтуер 

## Трениране на модел
Ще натренираме модел, който използва отворен dataset, за да оцени дали дадено ревю за филм е позитивно или негативно.

In [1]:
from datasets import load_dataset
dataset = "sepidmnorozy/Bulgarian_sentiment"
ds = load_dataset(dataset, split="train")
ds['text'][:5]

['Невероятен!!! Ако искате да гледате нещо нестандартно, не се двоумете - това е филмът за вас ;))',
 'Много добре направен :)',
 'Определено най-добрият Хобит!',
 'Муден старт, но нататък е хубав, ненатоварващ и приятен!',
 'Невероятна поредица.']

Първо, трябва да превърнем текста в ML-readable формат, тоест във вектор

In [2]:
from preprocessing import Tokenizer, TokenizingDataset
vocab_size = 350    # number of tokens
tokenizer = Tokenizer(vocab_size)
tokenizer.fit(ds['text'])
train_ds = TokenizingDataset(ds, tokenizer)
train_ds[0]

(tensor([348,   9, 102, 229,  97,  48, 311, 159,   2, 165, 116, 135, 139, 116,
         107, 184, 107,  67, 221, 286,  52,  11,  70,  71,  95, 136,  44, 214,
          31, 164, 116,  23,  43, 115, 142,  61, 216, 114,  83, 178,  50,  45,
          45, 347, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349,
         349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349,
         349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349],
        dtype=torch.int32),
 tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  

Тренираме модел

In [3]:
from training_procedure import fit_model
model = fit_model(train_ds)

epoch_loss=914.6326519995928
epoch accuracy: 0.7354028085735402
epoch_loss=978.675831027329
epoch accuracy: 0.8471914264597191
epoch_loss=1023.2758037447929
epoch accuracy: 0.8468218773096822
epoch_loss=961.9597607664764
epoch accuracy: 0.844789356984479
epoch_loss=961.1016171090305
epoch accuracy: 0.840169992609017
epoch_loss=952.2176993377507
epoch accuracy: 0.8377679231337768
epoch_loss=918.0857241936028
epoch accuracy: 0.8379526977087953
epoch_loss=937.9692882113159
epoch accuracy: 0.8457132298595713
epoch_loss=907.4984237905592
epoch accuracy: 0.8484848484848485
epoch_loss=919.4764760602266
epoch accuracy: 0.8519955654101996
epoch_loss=907.1320295222104
epoch accuracy: 0.8551367331855136
epoch_loss=884.2947474662215
epoch accuracy: 0.8593865484109386
epoch_loss=864.6668815985322
epoch accuracy: 0.8630820399113082
epoch_loss=888.9133908189833
epoch accuracy: 0.8623429416112343
epoch_loss=866.0947042088956
epoch accuracy: 0.8654841093865484
epoch_loss=860.8033702131361
epoch accurac

За да не се налага да тренираме всеки път, запазваме нашите тренирани компоненти

In [4]:
import torch
tokenizer.save("tokenizer.pkl")
torch.save(model, "model.pt")

За да използваме модела:

In [5]:
model(torch.IntTensor([[10, 15, 20, 25]]))

NameError: name 'torch' is not defined

## Inference

In [6]:
import torch
class SentimentInference():
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def __call__(self, inp: str):
        tokens = self.tokenizer.tokenize(inp)[:self.model.context_size+1]
        res = self.model(torch.IntTensor([tokens]))
        return res.argmax().item()
    
    @staticmethod
    def load():
        m = torch.load("model.pt")
        t = Tokenizer.load("tokenizer.pkl")
        return SentimentInference(m, t)
inference = SentimentInference.load()
inference("скучен и глупав филм")

0

## Отваряне на достъпа
Следва да пуснем този модел на прод. Нашето решение ще бъде:
- в облака
- безплатно
- достъпно

# Какво липсва в тази презентация?
- security - всеки може да достъпва нашата виртуална машина. Нямаме никаква защита против атака, примерно DDOS. Това е отговорна задача и е важно да се следват добри практики. Това е дискусия за друга презентация и от друг презентатор
- scale - моделът е прост и има само една инстанция. Обикновено се налага да имаме възможност да осигурим повече ресурси. Тук е също голяма дискусия, но е хубаво да имаме предвид колко трафик е необходимо да издържа - прекалено много неизползвани ресурси са чиста загуба на пари, прекалено малко означава, че няма да имаме сървиз 
- docker и kubernetes - реших да не усложнявам презентацията, но принципно добрата практика диктува, че нашия сървиз би влязъл в контейнер и би скалирал с kubernetes. На практика, няма да се налага да правим ръчните команди по теглене на репото, а нашия сървиз би бил по-secure и по-manageable откъм monitoring, logging, automated scaling, etc
- stress test - на какъв брой рекуести може да издържи нашия сървиз
- monitoring - как да разберем възможно най-бързо когато има проблем със сървиза или AWS
- networking - важна тема, но реших да не разводнявам дискусията. Добра идея е поне да фиксираме адреса на нашия сървиз, така че да не го викаме по IP